## 1. Setup

In [61]:
import sys
import os
import h5py
import gc
from glob import glob
from pathlib import Path
from functools import lru_cache
from collections import defaultdict
from fastprogress import progress_bar
from copy import deepcopy
from tqdm import tqdm

import numpy as np
import pandas as pd
import PIL
from PIL import Image, ExifTags
import cv2
import sqlite3

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import backend as K
tf.config.set_visible_devices([], 'GPU')

import torch
import torch.nn.functional as F
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import kornia
from kornia.feature import (
    LoFTR, LocalFeature, PassLAF, LAFOrienter, PatchDominantGradientOrientation, OriNet, LAFAffNetShapeEstimator,
    KeyNetDetector, LAFDescriptor, HardNet8, HyNet, TFeat, SOSNet, get_laf_center, DescriptorMatcher
)

from transformers import TFViTModel, ViTFeatureExtractor

import pycolmap

In [43]:
root = Path('/home/gunes/Desktop/Kaggle/image-matching-challenge-2023')
competition_dataset = root / 'data'

In [44]:
sys.path.append(str(root / 'venv' / 'lib' / 'python3.9' / 'site-packages' / 'SuperGluePretrainedNetwork'))
from models.matching import Matching

In [45]:
df = pd.read_csv(competition_dataset / 'sample_submission.csv')

if df.shape[0] != 3:
    # Enable submission mode and disable verbose
    submission = True
    verbose = False
    train_or_test_directory = competition_dataset / 'test'
else:
    # Disable submission mode and enable verbose
    submission = False
    verbose = True
    train_or_test_directory = competition_dataset / 'train'

if submission is False:
    df = pd.read_csv(competition_dataset / 'train' / 'train_labels.csv')
    
print(f'Dataset Shape: {df.shape}')

Dataset Shape: (327, 5)


In [46]:
def load_angle_detection_model():
    
    """
    Load angle detection model and processor
    
    Returns
    -------
    model: keras.engine.functional.Functional
        Angle detection model
        
    processor: transformers.ViTFeatureExtractor
        Image processor
    """
    
    vit = TFViTModel.from_pretrained(root / 'models' / 'vit-base-patch16-224')
    
    input_layer = Input(shape=(3, 224, 224))
    x = vit(input_layer)
    y = Dense(1, activation='linear')(x[-1])
    
    model = Model(input_layer, y)
    model.load_weights(root / 'models' / 'deep-image-orientation-angle-detection' / 'model-vit-ang-loss.h5')
    
    processor = ViTFeatureExtractor.from_pretrained(root / 'models' / 'vit-base-patch16-224')
    
    return model, processor


def detect_angle(image, model, processor):
    
    """
    Detect angle of a given image
    
    Parameters
    ----------
    image: numpy.ndarray of shape (height, width, channel)
        Image array
        
    model: keras.engine.functional.Functional
        Angle detection model
        
    processor: transformers.ViTFeatureExtractor
        Image processor
    
    Returns
    -------
    outputs: float
    """
    
    inputs = processor(images=[image], return_tensors='np')['pixel_values']
    with tf.device('/cpu:0'):
        outputs = model.predict(inputs, verbose=None)[0][0]
        
    return outputs


In [47]:
angle_detection_model, angle_detection_processor = load_angle_detection_model()

for idx, row in tqdm(df.iterrows(), total=df.shape[0]):

    image_path = train_or_test_directory / row['image_path']
    image = Image.open(image_path)
    image = np.array(Image.open(image_path))
    
    # Extract image dimensions and memory usage
    df.loc[idx, 'image_height'] = image.shape[0]
    df.loc[idx, 'image_width'] = image.shape[1]
    df.loc[idx, 'memory_usage'] = image.nbytes
    
    # Extract image orientation
    angle = detect_angle(
        image=image,
        model=angle_detection_model,
        processor=angle_detection_processor
    )
    df.loc[idx, 'angle'] = -angle


df['image_id'] = df['image_path'].apply(lambda x: str(x).split('/')[-1])
df['memory_usage'] /= (1024 ** 2)

df['image_height'] = df['image_height'].astype(np.uint16)
df['image_width'] = df['image_width'].astype(np.uint16)
df['memory_usage'] = df['memory_usage'].astype(np.float32)
df['angle'] = df['angle'].astype(np.float32)

del angle_detection_model, angle_detection_processor
gc.collect()
torch.cuda.empty_cache()
K.clear_session()

Some layers from the model checkpoint at /home/gunes/Desktop/Kaggle/image-matching-challenge-2023/models/vit-base-patch16-224 were not used when initializing TFViTModel: ['classifier']
- This IS expected if you are initializing TFViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFViTModel were not initialized from the model checkpoint at /home/gunes/Desktop/Kaggle/image-matching-challenge-2023/models/vit-base-patch16-224 and are newly initialized: ['vit/pooler/dense/kernel:0', 'vit/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%

## 2. Image Utilities

In [72]:
def get_global_desc(image_paths, model, device):
        
    config = resolve_data_config({}, model=model)
    transform = create_transform(**config)
    
    global_descs = []
    for i, img_fname_full in tqdm(enumerate(image_paths),total=len(image_paths)):
        key = os.path.splitext(os.path.basename(img_fname_full))[0]
        img = Image.open(img_fname_full).convert('RGB')
        img = transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            with torch.autocast(device_type=device.type, dtype=torch.float16):
                desc = model.forward_features(img.to(device)).mean(dim=(-1, 2))
            desc = desc.view(1, -1).detach().cpu().float()
            desc = F.normalize(desc, dim=1, p=2)
        global_descs.append(desc)
    global_descs = torch.cat(global_descs, dim=0)
    
    return global_descs


def create_image_pairs(image_paths, sim_th=0.6, min_pairs=20, exhaustive_if_less=20, device=torch.device('cuda')):
    
    n_images = len(image_paths)

    if n_images <= exhaustive_if_less:
        
        image_pair_indices = []
        # Create all possible image pairs from given list of image paths
        for i in range(len(image_paths)):
            for j in range(i + 1, len(image_paths)):
                image_pair_indices.append((i, j))

        return image_pair_indices
    
    else:

        model = timm.create_model(
            'tf_efficientnet_b7',
            pretrained=True
        )
        model = model.eval().to(image_matching_device)
        descs = get_global_desc(image_paths, model, device=device)
        dm = torch.cdist(descs, descs, p=2).numpy()
        mask = dm <= sim_th
        total = 0
        image_pair_indices = []
        ar = np.arange(n_images)
        already_there_set = []
        for st_idx in range(n_images - 1):
            mask_idx = mask[st_idx]
            to_match = ar[mask_idx]
            if len(to_match) < min_pairs:
                to_match = np.argsort(dm[st_idx])[:min_pairs]  
            for idx in to_match:
                if st_idx == idx:
                    continue
                if dm[st_idx, idx] < 1000:
                    image_pair_indices.append(tuple(sorted((st_idx, idx.item()))))
                    total += 1
                    
        image_pair_indices = sorted(list(set(image_pair_indices)))
        
        return image_pair_indices


def resize_with_aspect_ratio(image, longest_edge):

    """
    Resize image while preserving its aspect ratio

    Parameters
    ----------
    image: numpy.ndarray of shape (height, width, 3)
        Image array

    longest_edge: int
        Desired number of pixels on the longest edge

    Returns
    -------
    image: numpy.ndarray of shape (resized_height, resized_width, 3)
        Resized image array
    """

    height, width = image.shape[:2]
    scale = longest_edge / max(height, width)
    image = cv2.resize(image, dsize=(int(width * scale), int(height * scale)), interpolation=cv2.INTER_LANCZOS4)

    return image


def get_image_tensor(image_path_or_array, resize, resize_shape, resize_longest_edge, scale, grayscale):

    """
    Load image and preprocess it

    Parameters
    ----------
    image_path_or_array: str or numpy.ndarray of shape (height, width, 3)
        Image path or image array

    resize: bool
        Whether to resize the image or not

    resize_shape: tuple or int
        Tuple of image height and width or number of pixels for both height and width

    resize_longest_edge: bool
        Whether to resize the longest edge or not

    scale: bool
        Whether to scale image pixel values by max 8-bit pixel value or not

    grayscale: bool
        Whether to convert RGB image to grayscale or not

    Returns
    -------
    image: torch.Tensor of shape (1, 1 or 3, height, width)
        Image tensor
    """

    if isinstance(image_path_or_array, Path) or isinstance(image_path_or_array, str):
        # Read image from the given path if image_path_or_array is a path-like string
        image = cv2.imread(str(image_path_or_array))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
        image = image_path_or_array

    if resize:
        if resize_longest_edge:
            image = resize_with_aspect_ratio(image=image, longest_edge=resize_shape)
        else:
            resize_shape = (resize_shape, resize_shape) if isinstance(resize_shape, int) else resize_shape
            image = cv2.resize(image, resize_shape, interpolation=cv2.INTER_LANCZOS4)

    if scale:
        image = image / 255.

    image = kornia.image_to_tensor(image, False).float()
    if grayscale:
        image = kornia.color.rgb_to_grayscale(image)

    return image


def crop(image, keypoints, perc_points=0.85, pad=5):

    norm_keypoints = MinMaxScaler().fit_transform(keypoints)
    total = len(keypoints)
    best_dist = 1
    best_clusters = None
    best_asm = None
    for eps in [0.01, 0.025, 0.05, 0.1, 0.2]:
        clusters = DBSCAN(eps=eps).fit_predict(norm_keypoints)
        counts = pd.Series(clusters).value_counts().sort_values(ascending=False)
        counts = counts[counts.index > -1]
        if len(counts) == 0:
            continue

        cumsums = np.cumsum(counts.values) / total
        dists = np.abs(cumsums - perc_points)
        best_ix = np.argmin(dists)

        if dists[best_ix] < best_dist:
            best_dist = dists[best_ix]
            best_clusters = list(counts.head(best_ix + 1).index)
            best_asm = clusters

    mask = np.isin(best_asm, best_clusters)

    miny = int(np.min(keypoints[mask][:, 1]))
    miny = max(miny - pad, 0)

    maxy = int(np.max(keypoints[mask][:, 1]))
    maxy = min(maxy + pad, image.shape[0])

    minx = int(np.min(keypoints[mask][:, 0]))
    minx = max(minx - pad, 0)

    maxx = int(np.max(keypoints[mask][:, 0]))
    maxx = min(maxx + pad, image.shape[1])

    image_cropped = image[miny:maxy + 1, minx:maxx + 1, :]
    keypoints_cropped = np.copy(keypoints)
    keypoints_cropped[:, 0] -= minx
    keypoints_cropped[:, 1] -= miny

    keypoints_cropped = keypoints_cropped[(keypoints_cropped[:, 0] > 0) & (keypoints_cropped[:, 1] > 0) & (keypoints_cropped[:, 0] < image_cropped.shape[1]) & (keypoints_cropped[:, 1] < image_cropped.shape[0])]

    return image_cropped, keypoints_cropped, minx, miny


def rotate_image(img, angle):
    height, width = img.shape[:2]
    matrix = cv2.getRotationMatrix2D((width / 2 - 0.5, height / 2 - 0.5), angle, 1.0)
    matrix = np.vstack((matrix, [[0, 0, 1]]))
    return cv2.warpPerspective(img, matrix, (width, height))


def rotate_coordinates(img, angle, keypoints):
    height, width = img.shape[:2]
    inv_matrix = cv2.getRotationMatrix2D((width / 2 - 0.5, height / 2 - 0.5), angle, 1.0)
    inv_matrix = np.vstack((inv_matrix, [[0, 0, 1]]))
    return cv2.perspectiveTransform(keypoints[None, :, :], inv_matrix)[0]


## 3. Camera Utilities

In [49]:
def get_focal_length(image_path):

    """
    Get focal length from EXIF or calculate it using prior

    Parameters
    ----------
    image_path: str
        Image path

    Returns
    -------
    focal_length: float
        Focal length extracted from EXIF or calculated using prior
    """

    image = Image.open(image_path)
    image_longest_edge = max(image.size)

    focal_length = None
    exif = image.getexif()
    
    if image._getexif() is not None:
        exif = {
            ExifTags.TAGS[k]: v
            for k, v in image._getexif().items()
            if k in ExifTags.TAGS
        }
    else:
        exif = None

    if exif is not None:

        focal_length_35mm = None

        for tag, value in exif.items():
            if tag == 'FocalLengthIn35mmFilm':
                focal_length_35mm = float(value)
        
        if focal_length_35mm is not None:
            focal_length = focal_length_35mm / 35. * image_longest_edge

    if focal_length is None:
        prior_focal_length = 1.2
        focal_length = prior_focal_length * image_longest_edge

    return focal_length


## 4. Database Utilities

In [50]:
MAX_IMAGE_ID = 2 ** 31 - 1

CREATE_CAMERAS_TABLE = """CREATE TABLE IF NOT EXISTS cameras (
    camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    model INTEGER NOT NULL,
    width INTEGER NOT NULL,
    height INTEGER NOT NULL,
    params BLOB,
    prior_focal_length INTEGER NOT NULL)"""

CREATE_DESCRIPTORS_TABLE = """CREATE TABLE IF NOT EXISTS descriptors (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)"""

CREATE_IMAGES_TABLE = """CREATE TABLE IF NOT EXISTS images (
    image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name TEXT NOT NULL UNIQUE,
    camera_id INTEGER NOT NULL,
    prior_qw REAL,
    prior_qx REAL,
    prior_qy REAL,
    prior_qz REAL,
    prior_tx REAL,
    prior_ty REAL,
    prior_tz REAL,
    CONSTRAINT image_id_check CHECK(image_id >= 0 and image_id < {}),
    FOREIGN KEY(camera_id) REFERENCES cameras(camera_id))
""".format(MAX_IMAGE_ID)

CREATE_TWO_VIEW_GEOMETRIES_TABLE = """
CREATE TABLE IF NOT EXISTS two_view_geometries (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    config INTEGER NOT NULL,
    F BLOB,
    E BLOB,
    H BLOB)
"""

CREATE_KEYPOINTS_TABLE = """CREATE TABLE IF NOT EXISTS keypoints (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)
"""

CREATE_MATCHES_TABLE = """CREATE TABLE IF NOT EXISTS matches (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB)"""

CREATE_NAME_INDEX = \
    "CREATE UNIQUE INDEX IF NOT EXISTS index_name ON images(name)"

CREATE_ALL = "; ".join([
    CREATE_CAMERAS_TABLE,
    CREATE_IMAGES_TABLE,
    CREATE_KEYPOINTS_TABLE,
    CREATE_DESCRIPTORS_TABLE,
    CREATE_MATCHES_TABLE,
    CREATE_TWO_VIEW_GEOMETRIES_TABLE,
    CREATE_NAME_INDEX
])


def image_ids_to_pair_id(image_id1, image_id2):
    if image_id1 > image_id2:
        image_id1, image_id2 = image_id2, image_id1
    return image_id1 * MAX_IMAGE_ID + image_id2


def pair_id_to_image_ids(pair_id):
    image_id2 = pair_id % MAX_IMAGE_ID
    image_id1 = (pair_id - image_id2) / MAX_IMAGE_ID
    return image_id1, image_id2


def array_to_blob(array):
    return array.tobytes()


def blob_to_array(blob, dtype, shape=(-1,)):
    return np.fromstring(blob, dtype=dtype).reshape(*shape)


class COLMAPDatabase(sqlite3.Connection):

    @staticmethod
    def connect(database_path):
        return sqlite3.connect(database_path, factory=COLMAPDatabase)


    def __init__(self, *args, **kwargs):
        super(COLMAPDatabase, self).__init__(*args, **kwargs)

        self.create_tables = lambda: self.executescript(CREATE_ALL)
        self.create_cameras_table = \
            lambda: self.executescript(CREATE_CAMERAS_TABLE)
        self.create_descriptors_table = \
            lambda: self.executescript(CREATE_DESCRIPTORS_TABLE)
        self.create_images_table = \
            lambda: self.executescript(CREATE_IMAGES_TABLE)
        self.create_two_view_geometries_table = \
            lambda: self.executescript(CREATE_TWO_VIEW_GEOMETRIES_TABLE)
        self.create_keypoints_table = \
            lambda: self.executescript(CREATE_KEYPOINTS_TABLE)
        self.create_matches_table = \
            lambda: self.executescript(CREATE_MATCHES_TABLE)
        self.create_name_index = lambda: self.executescript(CREATE_NAME_INDEX)

    def add_camera(self, model, width, height, params,
                   prior_focal_length=False, camera_id=None):
        params = np.asarray(params, np.float64)
        cursor = self.execute(
            "INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)",
            (camera_id, model, width, height, array_to_blob(params),
             prior_focal_length))
        return cursor.lastrowid

    def add_image(self, name, camera_id,
                  prior_q=np.zeros(4), prior_t=np.zeros(3), image_id=None):
        cursor = self.execute(
            "INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (image_id, name, camera_id, prior_q[0], prior_q[1], prior_q[2],
             prior_q[3], prior_t[0], prior_t[1], prior_t[2]))
        return cursor.lastrowid

    def add_keypoints(self, image_id, keypoints):
        assert(len(keypoints.shape) == 2)
        assert(keypoints.shape[1] in [2, 4, 6])

        keypoints = np.asarray(keypoints, np.float32)
        self.execute(
            "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
            (image_id,) + keypoints.shape + (array_to_blob(keypoints),))

    def add_descriptors(self, image_id, descriptors):
        descriptors = np.ascontiguousarray(descriptors, np.uint8)
        self.execute(
            "INSERT INTO descriptors VALUES (?, ?, ?, ?)",
            (image_id,) + descriptors.shape + (array_to_blob(descriptors),))

    def add_matches(self, image_id1, image_id2, matches):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)

        if image_id1 > image_id2:
            matches = matches[:,::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        self.execute(
            "INSERT INTO matches VALUES (?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches),))

    def add_two_view_geometry(self, image_id1, image_id2, matches,
                              F=np.eye(3), E=np.eye(3), H=np.eye(3), config=2):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)

        if image_id1 > image_id2:
            matches = matches[:,::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        F = np.asarray(F, dtype=np.float64)
        E = np.asarray(E, dtype=np.float64)
        H = np.asarray(H, dtype=np.float64)
        self.execute(
            "INSERT INTO two_view_geometries VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches), config,
             array_to_blob(F), array_to_blob(E), array_to_blob(H)))

In [51]:
def create_camera(db, image_path, camera_model):
    
    image = Image.open(image_path)
    width, height = image.size

    focal_length = get_focal_length(image_path)

    if camera_model == 'simple-pinhole':
        model = 0
        param_arr = np.array([focal_length, width / 2, height / 2])
    if camera_model == 'pinhole':
        model = 1
        param_arr = np.array([focal_length, focal, width / 2, height / 2])
    elif camera_model == 'simple-radial':
        model = 2
        param_arr = np.array([focal_length, width / 2, height / 2, 0.1])
    elif camera_model == 'opencv':
        model = 4
        param_arr = np.array([focal_length, focal_length, width / 2, height / 2, 0., 0., 0., 0.])
         
    return db.add_camera(model, width, height, param_arr)


def add_keypoints(db, h5_path, image_path, camera_model, single_camera=True):
    
    keypoint_f = h5py.File(os.path.join(h5_path, 'keypoints.h5'), 'r')
    camera_id = None
    fname_to_id = {}
    
    for filename in tqdm(list(keypoint_f.keys())):
        
        keypoints = keypoint_f[filename][()]

        path = os.path.join(image_path, filename)
        if not os.path.isfile(path):
            raise IOError(f'Invalid image path {path}')

        if camera_id is None or not single_camera:
            camera_id = create_camera(db, path, camera_model)
        image_id = db.add_image(filename, camera_id)
        fname_to_id[filename] = image_id

        db.add_keypoints(image_id, keypoints)

    return fname_to_id


def add_matches(db, h5_path, fname_to_id):
    
    match_file = h5py.File(os.path.join(h5_path, 'matches.h5'), 'r')
    
    added = set()
    n_keys = len(match_file.keys())
    n_total = (n_keys * (n_keys - 1)) // 2

    with tqdm(total=n_total) as pbar:
        for key_1 in match_file.keys():
            group = match_file[key_1]
            for key_2 in group.keys():
                id_1 = fname_to_id[key_1]
                id_2 = fname_to_id[key_2]

                pair_id = image_ids_to_pair_id(id_1, id_2)
                if pair_id in added:
                    continue
            
                matches = group[key_2][()]
                db.add_matches(id_1, id_2, matches)
                added.add(pair_id)
                pbar.update(1)
                
                
def get_unique_idxs(A, dim=0):
    
    unique, idx, counts = torch.unique(A, dim=dim, sorted=True, return_inverse=True, return_counts=True)
    _, ind_sorted = torch.sort(idx, stable=True)
    cum_sum = counts.cumsum(0)
    cum_sum = torch.cat((torch.tensor([0],device=cum_sum.device), cum_sum[:-1]))
    first_indices = ind_sorted[cum_sum]
    
    return first_indices


## 5. Evaluation

In [52]:
def array_to_string(a):

    """
    Flatten given array and convert it to a string with semicolon delimiters

    Parameters
    ----------
    a: np.ndarray
        N-dimensional array

    Returns
    -------
    s: string
        String form of the given array
    """

    s = ';'.join([str(x) for x in a.reshape(-1)])

    return s


def string_to_array(s):

    """
    Convert semicolon delimited string to an array

    Parameters
    ----------
    s: string
        String form of the array

    Returns
    -------
    a: np.ndarray
        N-dimensional array
    """

    a = np.array(s.split(';')).astype(np.float64)

    return a


def rotation_matrix_to_quaternion(rotation_matrix):

    """
    Convert rotation matrix to quaternion

    Parameters
    ----------
    rotation_matrix: numpy.ndarray of shape (3, 3)
        Array of directions of the world-axes in camera coordinates

    Returns
    -------
    quaternion: numpy.ndarray of shape (4)
        Array of quaternion
    """

    r00 = rotation_matrix[0, 0]
    r01 = rotation_matrix[0, 1]
    r02 = rotation_matrix[0, 2]
    r10 = rotation_matrix[1, 0]
    r11 = rotation_matrix[1, 1]
    r12 = rotation_matrix[1, 2]
    r20 = rotation_matrix[2, 0]
    r21 = rotation_matrix[2, 1]
    r22 = rotation_matrix[2, 2]

    k = np.array([
        [r00 - r11 - r22, 0.0, 0.0, 0.0],
        [r01 + r10, r11 - r00 - r22, 0.0, 0.0],
        [r02 + r20, r12 + r21, r22 - r00 - r11, 0.0],
        [r21 - r12, r02 - r20, r10 - r01, r00 + r11 + r22]
    ])
    k /= 3.0

    # Quaternion is the eigenvector of k that corresponds to the largest eigenvalue
    w, v = np.linalg.eigh(k)
    quaternion = v[[3, 0, 1, 2], np.argmax(w)]

    if quaternion[0] < 0:
        np.negative(quaternion, quaternion)

    return quaternion


def pose_difference(r1, t1, r2, t2):

    """
    Calculate relative pose difference from given rotation matrices and translation vectors

    Parameters
    ----------
    r1: numpy.ndarray of shape (3, 3)
        First rotation matrix

    t1: numpy.ndarray of shape (3)
        First translation vector

    r2: numpy.ndarray of shape (3, 3)
        Second rotation matrix

    t2: numpy.ndarray of shape (3)
        Second translation vector

    Returns
    -------
    rotation_difference: float
        Rotation difference in terms of degrees from the first image

    translation_difference: float
        Translation difference in terms of meters from the first image
    """

    rotation_difference = np.dot(r2, r1.T)
    translation_difference = t2 - np.dot(rotation_difference, t1)

    return rotation_difference, translation_difference


def rotation_and_translation_error(q_ground_truth, t_ground_truth, q_prediction, t_prediction, epsilon=1e-15):

    """
    Calculate rotation and translation error

    Parameters
    ----------
    q_ground_truth: numpy.ndarray of shape (4)
        Array of quaternion derived from ground truth rotation matrix

    t_ground_truth: numpy.ndarray of shape (3)
        Array of ground truth translation vector

    q_prediction: numpy.ndarray of shape (4)
        Array of quaternion derived from estimated rotation matrix

    t_prediction: numpy.ndarray of shape (3)
        Array of estimated translation vector

    epsilon: float
        A small number for preventing zero division

    Returns
    -------
    rotation_error: float
        Rotation error in terms of degrees

    translation_error: float
        Translation error in terms of meters
    """

    q_ground_truth_norm = q_ground_truth / (np.linalg.norm(q_ground_truth) + epsilon)
    q_prediction_norm = q_prediction / (np.linalg.norm(q_prediction) + epsilon)
    loss_q = np.maximum(epsilon, (1.0 - np.sum(q_prediction_norm * q_ground_truth_norm) ** 2))

    rotation_error = np.degrees(np.arccos(1 - (2 * loss_q)))

    scaling_factor = np.linalg.norm(t_ground_truth)
    t_prediction = scaling_factor * (t_prediction / (np.linalg.norm(t_prediction) + epsilon))
    translation_error = min(
        np.linalg.norm(t_ground_truth - t_prediction),
        np.linalg.norm(t_ground_truth + t_prediction)
    )

    return rotation_error, translation_error


def mean_average_accuracy(rotation_errors, translation_errors, rotation_error_thresholds, translation_error_thresholds):

    """
    Calculate mean average accuracies over a set of thresholds for rotation and translation

    Parameters
    ----------
    rotation_errors: list of shape (n_pairs)
        List of rotation errors

    translation_errors: list of shape (n_pairs)
        List of translation errors

    rotation_error_thresholds: numpy.ndarray of shape (10)
        Array of rotation error thresholds

    translation_error_thresholds: numpy.ndarray of shape (10)
        Array of translation error thresholds

    Returns
    -------
    maa: float
        Mean average accuracy calculated on both rotation and translation errors

    rotation_maa: float
        Mean average accuracy calculated on rotation errors

    translation_maa: float
        Mean average accuracy calculated on translation errors
    """

    accuracies, rotation_accuracies, translation_accuracies = [], [], []

    for rotation_error_threshold, translation_error_threshold in zip(rotation_error_thresholds, translation_error_thresholds):

        # Calculate whether the errors are less than specified thresholds or not
        rotation_accuracy = (rotation_errors <= rotation_error_threshold)
        translation_accuracy = (translation_errors <= translation_error_threshold)
        accuracy = rotation_accuracy & translation_accuracy

        accuracies.append(accuracy.astype(np.float32).mean())
        rotation_accuracies.append(rotation_accuracy.astype(np.float32).mean())
        translation_accuracies.append(translation_accuracy.astype(np.float32).mean())

    maa = np.array(accuracies).mean()
    rotation_maa = np.array(rotation_accuracies).mean()
    translation_maa = np.array(translation_accuracies).mean()

    return maa, rotation_maa, translation_maa


def evaluate(df, verbose=False):

    """
    Calculate mean average accuracies over a set of thresholds for rotation and translation

    Parameters
    ----------
    df: pandas.DataFrame
        Dataframe with dataset, scene, rotation_matrix, translation_vector, rotation_matrix_prediction and translation_vector_prediction columns

    verbose: bool
        Whether to print scores or not

    Returns
    -------
    df_scores: pandas.DataFrame
        Dataframe of scores
    """

    rotation_error_thresholds = {
        **{('haiper', scene): np.linspace(1, 10, 10) for scene in ['bike', 'chairs', 'fountain']},
        **{('heritage', scene): np.linspace(1, 10, 10) for scene in ['cyprus', 'dioscuri']},
        **{('heritage', 'wall'): np.linspace(0.2, 10, 10)},
        **{('urban', 'kyiv-puppet-theater'): np.linspace(1, 10, 10)},
    }
    translation_error_thresholds = {
        **{('haiper', scene): np.geomspace(0.05, 0.5, 10) for scene in ['bike', 'chairs', 'fountain']},
        **{('heritage', scene): np.geomspace(0.1, 2, 10) for scene in ['cyprus', 'dioscuri']},
        **{('heritage', 'wall'): np.geomspace(0.05, 1, 10)},
        **{('urban', 'kyiv-puppet-theater'): np.geomspace(0.5, 5, 10)},
    }
    df_scores = pd.DataFrame(columns=['dataset', 'scene', 'image_pairs', 'maa', 'rotation_maa', 'translation_maa'])

    for (dataset, scene), df_scene in tqdm(df.groupby(['dataset', 'scene'])):

        scene_rotation_errors = []
        scene_translation_errors = []

        for i in range(df_scene.shape[0]):
            for j in range(i + 1, df_scene.shape[0]):

                rotation_matrix_difference_ground_truth, translation_vector_difference_ground_truth = pose_difference(
                    r1=string_to_array((df_scene.iloc[i]['rotation_matrix'])).reshape(3, 3),
                    t1=string_to_array((df_scene.iloc[i]['translation_vector'])),
                    r2=string_to_array((df_scene.iloc[j]['rotation_matrix'])).reshape(3, 3),
                    t2=string_to_array((df_scene.iloc[j]['translation_vector'])),
                )
                quaternion_ground_truth = rotation_matrix_to_quaternion(rotation_matrix=rotation_matrix_difference_ground_truth)

                rotation_matrix_difference_prediction, translation_vector_difference_prediction = pose_difference(
                    r1=string_to_array((df_scene.iloc[i]['rotation_matrix_prediction'])).reshape(3, 3),
                    t1=string_to_array((df_scene.iloc[i]['translation_vector_prediction'])),
                    r2=string_to_array((df_scene.iloc[j]['rotation_matrix_prediction'])).reshape(3, 3),
                    t2=string_to_array((df_scene.iloc[j]['translation_vector_prediction'])),
                )
                quaternion_prediction = rotation_matrix_to_quaternion(rotation_matrix=rotation_matrix_difference_prediction)

                rotation_error, translation_error = rotation_and_translation_error(
                    q_ground_truth=quaternion_ground_truth,
                    t_ground_truth=translation_vector_difference_ground_truth,
                    q_prediction=quaternion_prediction,
                    t_prediction=translation_vector_difference_prediction,
                    epsilon=1e-15
                )
                scene_rotation_errors.append(rotation_error)
                scene_translation_errors.append(translation_error)

        scene_maa, scene_rotation_maa, scene_translation_maa = mean_average_accuracy(
            rotation_errors=scene_rotation_errors,
            translation_errors=scene_translation_errors,
            rotation_error_thresholds=rotation_error_thresholds[(dataset, scene)],
            translation_error_thresholds=translation_error_thresholds[(dataset, scene)]
        )

        if verbose:
            settings.logger.info(
                f'''
                Dataset: {dataset} - Scene: {scene}
                Number of image pairs: {len(scene_rotation_errors)}
                mAA: {scene_maa:.6f} - rotation mAA: {scene_rotation_maa:.6f} - translation mAA: {scene_translation_maa:.6f}
                '''
            )

        df_scores = pd.concat((
            df_scores,
            pd.DataFrame(
                data=[[dataset, scene, len(scene_rotation_errors), scene_maa, scene_rotation_maa, scene_translation_maa]],
                columns=['dataset', 'scene', 'image_pairs', 'maa', 'rotation_maa', 'translation_maa']
            )
        ), axis=0)

    return df_scores


## 6. Models

In [53]:
def loftr_match_images(image1, image2, model, device, amp, transforms, confidence_threshold, top_k):

    """
    Match given two images with each other using LoFTR model

    Parameters
    ----------
    image1: numpy.ndarray of shape (3, height, width)
        Array of first image

    image2: numpy.ndarray of shape (3, height, width)
        Array of second image

    model: torch.nn.Module
        LoFTR Model

    device: torch.device
        Location of the image1, image2 and the model

    amp: bool
        Whether to use auto mixed precision or not

    transforms: dict
        Dictionary of transform parameters

    confidence_threshold: float or int
        Confidence threshold to filter out low confidence matches

    top_k: int
        Number of matches to take

    Returns
    -------
    outputs: dict
        Model outputs
    """

    image1_raw_height, image1_raw_width = image1.shape[:2]
    image1 = get_image_tensor(
        image_path_or_array=image1,
        resize=transforms['resize'],
        resize_shape=transforms['resize_shape'],
        resize_longest_edge=transforms['resize_longest_edge'],
        scale=transforms['scale'],
        grayscale=transforms['grayscale']
    )
    image1 = image1.to(device)
    image1_transformed_height, image1_transformed_width = image1.shape[2:]

    image2_raw_height, image2_raw_width = image2.shape[:2]
    image2 = get_image_tensor(
        image_path_or_array=image2,
        resize=transforms['resize'],
        resize_shape=transforms['resize_shape'],
        resize_longest_edge=transforms['resize_longest_edge'],
        scale=transforms['scale'],
        grayscale=transforms['grayscale']
    )
    image2 = image2.to(device)
    image2_transformed_height, image2_transformed_width = image2.shape[2:]

    with torch.no_grad():
        if amp:
            with torch.autocast(device_type=device.type, dtype=torch.float16):
                outputs = model({'image0': image1, 'image1': image2})
        else:
            outputs = model({'image0': image1, 'image1': image2})

    for k in outputs.keys():
        outputs[k] = outputs[k].detach().cpu().numpy()

    if confidence_threshold is not None:
        if isinstance(confidence_threshold, float):
            # Select matched keypoints with above given confidence threshold
            confidence_mask = outputs['confidence'] >= confidence_threshold
        elif isinstance(confidence_threshold, int):
            # Select keypoints dynamically based on confidence distribution
            confidence_mean, confidence_std = outputs['confidence'].mean(), outputs['confidence'].std()
            confidence_mask = outputs['confidence'] >= (confidence_mean + (confidence_std * confidence_threshold))
        else:
            raise ValueError(f'Invalid confidence_threshold {confidence_threshold}')

        for k in outputs.keys():
            outputs[k] = outputs[k][confidence_mask]

    if top_k is not None:
        # Select top-k keypoints based on their confidences
        sorting_idx = outputs['matching_scores0'].argsort()[-top_k:]
        for k in outputs.keys():
            outputs[k] = outputs[k][sorting_idx]

    outputs['keypoints0'][:, 0] *= image1_raw_width / image1_transformed_width
    outputs['keypoints0'][:, 1] *= image1_raw_height / image1_transformed_height
    outputs['keypoints1'][:, 0] *= image2_raw_width / image2_transformed_width
    outputs['keypoints1'][:, 1] *= image2_raw_height / image2_transformed_height

    return outputs


In [54]:
def superglue_match_images(image1, image2, model, device, amp, transforms, score_threshold, top_k):

    """
    Match given two images with each other using SuperGlue model

    Parameters
    ----------
    image1: numpy.ndarray of shape (3, height, width)
        Array of first image

    image2: numpy.ndarray of shape (3, height, width)
        Array of second image

    model: torch.nn.Module
        SuperGlue Model

    device: torch.device
        Location of the image1, image2 and the model

    amp: bool
        Whether to use auto mixed precision or not

    transforms: dict
        Dictionary of transform parameters

    score_threshold: float, int or None
        Confidence threshold

    top_k: int or None
        Number of keypoints to take

    Returns
    -------
    outputs: dict
        Model outputs
    """

    image1_raw_height, image1_raw_width = image1.shape[:2]
    image1 = get_image_tensor(
        image_path_or_array=image1,
        resize=transforms[1]['resize'],
        resize_shape=transforms[1]['resize_shape'],
        resize_longest_edge=transforms[1]['resize_longest_edge'],
        scale=transforms[1]['scale'],
        grayscale=transforms[1]['grayscale']
    )
    image1 = image1.to(device)
    image1_transformed_height, image1_transformed_width = image1.shape[2:]

    image2_raw_height, image2_raw_width = image2.shape[:2]
    image2 = get_image_tensor(
        image_path_or_array=image2,
        resize=transforms[2]['resize'],
        resize_shape=transforms[2]['resize_shape'],
        resize_longest_edge=transforms[2]['resize_longest_edge'],
        scale=transforms[2]['scale'],
        grayscale=transforms[2]['grayscale']
    )
    image2 = image2.to(device)
    image2_transformed_height, image2_transformed_width = image2.shape[2:]
    
    with torch.no_grad():
        if amp:
            with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
                outputs = model({'image0': image1, 'image1': image2})
        else:
            outputs = model({'image0': image1, 'image1': image2})

    for k in outputs.keys():
        if k == 'descriptors0' or k == 'descriptors1':
            outputs[k] = outputs[k][0].detach().cpu().numpy().T
        else:
            outputs[k] = outputs[k][0].detach().cpu().numpy()

    matches_mask = outputs['matches0'] > -1

    for k in ['keypoints1', 'scores1', 'descriptors1', 'matches1', 'matching_scores1']:
        outputs[k] = outputs[k][outputs['matches0'][matches_mask]]

    for k in ['keypoints0', 'scores0', 'descriptors0', 'matches0', 'matching_scores0']:
        outputs[k] = outputs[k][matches_mask]

    if score_threshold is not None:
        if isinstance(score_threshold, float):
            # Select matched keypoints with above given score threshold
            score_mask = outputs['matching_scores0'] >= score_threshold
        elif isinstance(score_threshold, int):
            # Select keypoints dynamically based on score distribution
            score_mean, score_std = outputs['matching_scores0'].mean(), outputs['matching_scores0'].std()
            score_mask = outputs['matching_scores0'] >= (score_mean + (score_std * score_threshold))
        else:
            raise ValueError(f'Invalid score_threshold {score_threshold}')

        for k in outputs.keys():
            outputs[k] = outputs[k][score_mask]

    if top_k is not None:
        # Select top-k keypoints based on their scores
        sorting_idx = outputs['matching_scores0'].argsort()[-top_k:]
        for k in outputs.keys():
            outputs[k] = outputs[k][sorting_idx]

    outputs['keypoints0'][:, 0] *= image1_raw_width / image1_transformed_width
    outputs['keypoints0'][:, 1] *= image1_raw_height / image1_transformed_height
    outputs['keypoints1'][:, 0] *= image2_raw_width / image2_transformed_width
    outputs['keypoints1'][:, 1] *= image2_raw_height / image2_transformed_height

    return outputs


## 7. Inference

In [62]:
image_matching_device = torch.device('cuda')

# Load LoFTR model with specified configurations
loftr_model = LoFTR(
    pretrained=None,
    config={
        'backbone_type': 'ResNetFPN',
        'resolution': (8, 2),
        'fine_window_size': 5,
        'fine_concat_coarse_feat': True,
        'resnetfpn': {
            'initial_dim': 128,
            'block_dims': [128, 196, 256]
        },
        'coarse': {
            'd_model': 256,
            'd_ffn': 256,
            'nhead': 8,
            'layer_names': ['self', 'cross', 'self', 'cross', 'self', 'cross', 'self', 'cross'],
            'attention': 'linear',
            'temp_bug_fix': False,
        },
        'match_coarse': {
            'thr': 0.2,
            'border_rm': 2,
            'match_type': 'dual_softmax',
            'dsmax_temperature': 0.1,
            'skh_iters': 3,
            'skh_init_bin_score': 1.0,
            'skh_prefilter': True,
            'train_coarse_percent': 0.4,
            'train_pad_num_gt_min': 200,
        },
        'fine': {
            'd_model':128,
            'd_ffn': 128,
            'nhead': 8,
            'layer_names': ['self', 'cross'],
            'attention': 'linear'
        }
    }
)
loftr_model.load_state_dict(torch.load(root / 'models' / 'loftr' / 'loftr_outdoor.ckpt')['state_dict'], strict=False)
loftr_model = loftr_model.eval().to(image_matching_device)

# Load SuperPoint and SuperGlue model with specified configurations
superglue_model = Matching(config={
    'superpoint': {
        'descriptor_dim': 256,
        'nms_radius': 4,
        'keypoint_threshold': 0.01,
        'max_keypoints': -1,
        'remove_borders': 4
    },
    'superglue': {
        'descriptor_dim': 256,
        'weights': 'outdoor',
        'keypoint_encoder': [32, 64, 128, 256],
        'sinkhorn_iterations': 100,
        'match_threshold': 0.2
    }
})
superglue_model = superglue_model.eval().to(image_matching_device)

Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)


In [66]:
def match(read_image_function, image_paths, image_pair_indices, feature_dir, loftr_model, superglue_model):
    
    with h5py.File(f'{feature_dir}/matches_loftr.h5', mode='w') as f_match:
        for pair_idx in progress_bar(image_pair_indices):
            
            idx1, idx2 = pair_idx
            fname1, fname2 = image_paths[idx1], image_paths[idx2]
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
            
            image1 = read_image_function(fname1)
            image2 = read_image_function(fname2)
            
            keypoints1 = []
            keypoints2 = []
            
            # Largest SuperGlue size that can be used on Kaggle GPU is 2560
            superglue_longest_edge_limit = 2560
            if (np.max(image1.shape[:2]) > superglue_longest_edge_limit) and (np.max(image2.shape[:2]) > superglue_longest_edge_limit):
                # Both of the images have longest edges greater than 2560
                first_stage_superglue_transforms = {
                    1: {
                        'resize': True,
                        'resize_shape': superglue_longest_edge_limit,
                        'resize_longest_edge': True,
                        'scale': True,
                        'grayscale': True
                    },
                    2: {
                        'resize': True,
                        'resize_shape': superglue_longest_edge_limit,
                        'resize_longest_edge': True,
                        'scale': True,
                        'grayscale': True
                    }
                }
            elif (np.max(image1.shape[:2]) > superglue_longest_edge_limit) and (np.max(image2.shape[:2]) <= superglue_longest_edge_limit):
                # First image's longest edge is greater than 2560
                first_stage_superglue_transforms = {
                    1: {
                        'resize': True,
                        'resize_shape': superglue_longest_edge_limit,
                        'resize_longest_edge': True,
                        'scale': True,
                        'grayscale': True
                    },
                    2: {
                        'resize': False,
                        'resize_shape': None,
                        'resize_longest_edge': None,
                        'scale': True,
                        'grayscale': True
                    }
                }
            elif (np.max(image1.shape[:2]) <= superglue_longest_edge_limit) and (np.max(image2.shape[:2]) > superglue_longest_edge_limit):
                # Second image's longest edge is greater than 2560
                first_stage_superglue_transforms = {
                    1: {
                        'resize': False,
                        'resize_shape': None,
                        'resize_longest_edge': None,
                        'scale': True,
                        'grayscale': True
                    },
                    2: {
                        'resize': True,
                        'resize_shape': superglue_longest_edge_limit,
                        'resize_longest_edge': True,
                        'scale': True,
                        'grayscale': True
                    }
                }
            else:
                # Neither of the image's longest edge is greater than 2560
                first_stage_superglue_transforms = {
                    1: {
                        'resize': False,
                        'resize_shape': None,
                        'resize_longest_edge': None,
                        'scale': True,
                        'grayscale': True
                    },
                    2: {
                        'resize': False,
                        'resize_shape': None,
                        'resize_longest_edge': None,
                        'scale': True,
                        'grayscale': True
                    }
                }
                
            
            first_stage_superglue_outputs = superglue_match_images(
                image1=image1,
                image2=image2,
                model=superglue_model,
                device=image_matching_device,
                amp=True,
                transforms=first_stage_superglue_transforms,
                score_threshold=None,
                top_k=None
            )
            keypoints1.append(first_stage_superglue_outputs['keypoints0'])
            keypoints2.append(first_stage_superglue_outputs['keypoints1'])
            
            '''
            loftr_outputs = loftr_match_images(
                image1=image1,
                image2=image2,
                model=loftr_model,
                device=image_matching_device,
                amp=True,
                transforms={
                    'resize': True,
                    'resize_shape': 840,
                    'resize_longest_edge': True,
                    'scale': True,
                    'grayscale': True,
                },
                confidence_threshold=None,
                top_k=None
            )
            first_stage_keypoints1.append(loftr_outputs['keypoints0'])
            first_stage_keypoints2.append(loftr_outputs['keypoints1'])
            '''
            
            keypoints1 = np.concatenate(keypoints1, axis=0)
            keypoints2 = np.concatenate(keypoints2, axis=0)
            
            stage2 = False
            if stage2:
                if keypoints1.shape[0] > 2:
                    
                    image1_cropped, keypoints1_cropped, x_offset1, y_offset1 = crop(image1, keypoints1)
                    image2_cropped, keypoints2_cropped, x_offset2, y_offset2 = crop(image2, keypoints2)

                    second_stage_superglue_transforms = {
                        1: {
                            'resize': True,
                            'resize_shape': 840,
                            'resize_longest_edge': True,
                            'scale': True,
                            'grayscale': True
                        },
                        2: {
                            'resize': True,
                            'resize_shape': 840,
                            'resize_longest_edge': True,
                            'scale': True,
                            'grayscale': True
                        }
                    }

                    second_stage_superglue_outputs = superglue_match_images(
                        image1=image1_cropped,
                        image2=image2_cropped,
                        model=superglue_model,
                        device=image_matching_device,
                        amp=True,
                        transforms=second_stage_superglue_transforms,
                        score_threshold=None,
                        top_k=None
                    )

                    second_stage_superglue_outputs['keypoints0'][:, 0] += x_offset1
                    second_stage_superglue_outputs['keypoints0'][:, 1] += y_offset1
                    second_stage_superglue_outputs['keypoints1'][:, 0] += x_offset2
                    second_stage_superglue_outputs['keypoints1'][:, 1] += y_offset2

                    keypoints1 = np.concatenate([keypoints1, second_stage_superglue_outputs['keypoints0']], axis=0)
                    keypoints2 = np.concatenate([keypoints2, second_stage_superglue_outputs['keypoints1']], axis=0)
            
            n_matches = len(keypoints1)
            group  = f_match.require_group(key1)
            if n_matches >= 5:
                group.create_dataset(key2, data=np.concatenate([keypoints1, keypoints2], axis=1))

    kpts = defaultdict(list)
    match_indexes = defaultdict(dict)
    total_kpts=defaultdict(int)
    
    with h5py.File(f'{feature_dir}/matches_loftr.h5', mode='r') as f_match:
        for k1 in f_match.keys():
            group  = f_match[k1]
            for k2 in group.keys():
                matches = group[k2][...]
                total_kpts[k1]
                kpts[k1].append(matches[:, :2])
                kpts[k2].append(matches[:, 2:])
                current_match = torch.arange(len(matches)).reshape(-1, 1).repeat(1, 2)
                current_match[:, 0]+=total_kpts[k1]
                current_match[:, 1]+=total_kpts[k2]
                total_kpts[k1]+=len(matches)
                total_kpts[k2]+=len(matches)
                match_indexes[k1][k2]=current_match

    for k in kpts.keys():
        kpts[k] = np.round(np.concatenate(kpts[k], axis=0))
        
    unique_kpts = {}
    unique_match_idxs = {}
    out_match = defaultdict(dict)
    
    for k in kpts.keys():
        uniq_kps, uniq_reverse_idxs = torch.unique(torch.from_numpy(kpts[k]),dim=0, return_inverse=True)
        unique_match_idxs[k] = uniq_reverse_idxs
        unique_kpts[k] = uniq_kps.numpy()
        
    for k1, group in match_indexes.items():
        for k2, m in group.items():
            m2 = deepcopy(m)
            m2[:,0] = unique_match_idxs[k1][m2[:,0]]
            m2[:,1] = unique_match_idxs[k2][m2[:,1]]
            mkpts = np.concatenate([unique_kpts[k1][ m2[:,0]],
                                    unique_kpts[k2][  m2[:,1]],
                                   ],
                                   axis=1)
            unique_idxs_current = get_unique_idxs(torch.from_numpy(mkpts), dim=0)
            m2_semiclean = m2[unique_idxs_current]
            unique_idxs_current1 = get_unique_idxs(m2_semiclean[:, 0], dim=0)
            m2_semiclean = m2_semiclean[unique_idxs_current1]
            unique_idxs_current2 = get_unique_idxs(m2_semiclean[:, 1], dim=0)
            m2_semiclean2 = m2_semiclean[unique_idxs_current2]
            out_match[k1][k2] = m2_semiclean2.numpy()
            
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp:
        for k, kpts1 in unique_kpts.items():
            f_kp[k] = kpts1
    
    with h5py.File(f'{feature_dir}/matches.h5', mode='w') as f_match:
        for k1, gr in out_match.items():
            group  = f_match.require_group(k1)
            for k2, match in gr.items():
                group[k2] = match
                
    return


def import_into_colmap(img_dir, feature_dir='.featureout', database_path = 'colmap.db'):
    
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, 'simple-radial', single_camera)
    add_matches(db, feature_dir, fname_to_id,)
    db.commit()
    
    return

In [71]:
reconstruction_root_directory = Path('./inference')
reconstruction_root_directory.mkdir(parents=True, exist_ok=True)

validation_scenes = {
    #'haiper': ['bike', 'chairs', 'fountain'],
    #'heritage': ['dioscuri', 'cyprus', 'wall'],
    'heritage': ['dioscuri'],
    #'urban': ['kyiv-puppet-theater']
}

for dataset in validation_scenes.keys():
    
    dataset_directory = competition_dataset / 'train' / dataset
    
    for scene in validation_scenes[dataset]:
        
        df_scene = df.loc[df['scene'] == scene]
        
        # Select LRU cache max size based on image mean memory usage
        scene_mean_memory_usage = df_scene['memory_usage'].mean()
        if scene_mean_memory_usage <= 8:
            lru_cache_max_size = 32
        else:
            lru_cache_max_size = 2
            
        @lru_cache(maxsize=lru_cache_max_size)
        def read_image(image_path):
            return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        
        scene_directory = dataset_directory / scene
        scene_image_directory = scene_directory / 'images'
        image_paths = sorted(glob(str(scene_image_directory / '*')))
        scene_image_count = len(image_paths)
        
        scene_reconstruction_directory = reconstruction_root_directory / dataset / scene
        scene_reconstruction_directory.mkdir(parents=True, exist_ok=True)                    
        database_path = scene_reconstruction_directory / 'colmap.db'
        if os.path.isfile(database_path):
            os.remove(database_path)
                    
        # Create image pair indices from given image paths
        scene_image_pair_indices = create_image_pairs(
            image_paths,
            sim_th=0.6,
            min_pairs=50,
            exhaustive_if_less=20,
            device=torch.device('cuda')
        )
            
        print(
            f'''
            Dataset: {dataset} - Scene: {scene}
            Image count: {scene_image_count} - Pair count: {len(scene_image_pair_indices)}
            Average memory usage per image: {scene_mean_memory_usage:.4f} - LRU cache max size: {lru_cache_max_size}
            '''
        )
        
        '''
        scene_mean_memory_usage_limit = 16
        if scene_mean_memory_usage < scene_mean_memory_usage_limit:
            match(
                read_image,
                image_paths,
                scene_image_pair_indices,
                feature_dir=scene_reconstruction_directory,
                loftr_model=loftr_model,
                superglue_model=superglue_model
            )
            import_into_colmap(scene_image_directory, feature_dir=scene_reconstruction_directory, database_path=database_path)
        else:
            sift_extraction_options = pycolmap.SiftExtractionOptions()
            sift_extraction_options.num_threads = -1
            sift_extraction_options.max_image_size = 1400
            sift_extraction_options.max_num_features = 8192
            sift_extraction_options.estimate_affine_shape = False
            sift_extraction_options.upright = False
            sift_extraction_options.normalization = 'L2'

            pycolmap.extract_features(
                database_path=database_path,
                image_path=scene_image_directory,
                image_list=[image_path.split('/')[-1] for image_path in image_paths],
                sift_options=sift_extraction_options,
                device=pycolmap.Device('cpu'),
                verbose=verbose
            )
        '''
        match(
            read_image,
            image_paths,
            scene_image_pair_indices,
            feature_dir=scene_reconstruction_directory,
            loftr_model=loftr_model,
            superglue_model=superglue_model
        )
        import_into_colmap(scene_image_directory, feature_dir=scene_reconstruction_directory, database_path=database_path)

        pycolmap.match_exhaustive(database_path)
        
        output_path = scene_reconstruction_directory / 'colmap_rec'
        output_path.mkdir(parents=True, exist_ok=True)

        incremental_mapper_options = pycolmap.IncrementalMapperOptions()
        incremental_mapper_options.min_model_size = 3
        incremental_mapper_options.min_num_matches = 5
        reconstructions = pycolmap.incremental_mapping(
            database_path=database_path,
            image_path=scene_image_directory,
            output_path=output_path,
            options=incremental_mapper_options
        )
    
        if len(reconstructions) > 0:

            best_registered_image_count = 0
            best_reconstruction_idx = None

            for reconstruction_idx in reconstructions.keys():
                if reconstructions[reconstruction_idx].num_reg_images() > best_registered_image_count:
                    best_reconstruction_idx = reconstruction_idx
                    best_registered_image_count = reconstructions[reconstruction_idx].num_reg_images()

            best_reconstruction = reconstructions[best_reconstruction_idx]
        else:
            best_registered_image_count = 0
            best_reconstruction_idx = None
            best_reconstruction = None

        print(
            f'''
            Dataset: {dataset} - Scene: {scene}
            Reconstruction count: {len(reconstructions)}
            Best reconstruction registered image count: {best_registered_image_count}/{len(image_paths)}
            '''
        )

        if best_reconstruction is not None:
            registered_images = {image.name: image for image in best_reconstruction.images.values()}
        else:
            registered_images = {}

        for idx, row in df.loc[df['scene'] == scene].iterrows():
            if row['image_id'] in registered_images:
                rotation_matrix_prediction = deepcopy(registered_images[row['image_id']].rotmat())
                translation_vector_prediction = deepcopy(registered_images[row['image_id']].tvec)
                df.loc[idx, 'rotation_matrix_prediction'] = ';'.join([str(x) for x in rotation_matrix_prediction.reshape(-1)])
                df.loc[idx, 'translation_vector_prediction'] = ';'.join([str(x) for x in translation_vector_prediction.reshape(-1)])
            else:
                df.loc[idx, 'rotation_matrix_prediction'] = np.nan
                df.loc[idx, 'translation_vector_prediction'] = np.nan

        # Fill unregistered images rotation matrices with the prediction mean or zeros
        scene_rotation_matrix_predictions = df.loc[df['scene'] == scene, 'rotation_matrix_prediction'].dropna().apply(lambda x: np.array(str(x).split(';'), dtype=np.float64).reshape(1, 3, 3)).values
        if scene_rotation_matrix_predictions.shape[0] == 0:
            rotation_matrix_fill_value = np.zeros((3, 3))
        else:
            rotation_matrix_fill_value = np.mean(np.concatenate(scene_rotation_matrix_predictions, axis=0), axis=0)
        df.loc[(df['scene'] == scene) & (df['rotation_matrix_prediction'].isnull()), 'rotation_matrix_prediction'] = ';'.join([str(x) for x in rotation_matrix_fill_value.reshape(-1)])

        # Fill unregistered images translation vectors with the prediction mean or zeros
        scene_translation_vector_predictions = df.loc[df['scene'] == scene, 'translation_vector_prediction'].dropna().apply(lambda x: np.array(str(x).split(';'), dtype=np.float64).reshape(1, 3)).values
        if scene_translation_vector_predictions.shape[0] == 0:
            translation_vector_fill_value = np.zeros((3, 1))
        else:
            translation_vector_fill_value = np.mean(np.concatenate(scene_translation_vector_predictions, axis=0), axis=0)
        df.loc[(df['scene'] == scene) & (df['translation_vector_prediction'].isnull()), 'translation_vector_prediction'] = ';'.join([str(x) for x in translation_vector_fill_value.reshape(-1)])


100%|█████████████████████████████████████████| 174/174 [00:08<00:00, 19.63it/s]


            Dataset: heritage - Scene: dioscuri
            Image count: 174 - Pair count: 5646
            Average memory usage per image: 2.1643 - LRU cache max size: 32
            


 33%|███████████▊                        | 4776/14535 [00:00<00:01, 7688.53it/s]



Exhaustive feature matching

Matching block [1/4, 1/4] in 0.903s
Matching block [1/4, 2/4] in 0.143s
Matching block [1/4, 3/4] in 0.310s
Matching block [1/4, 4/4] in 0.101s
Matching block [2/4, 1/4] in 0.215s
Matching block [2/4, 2/4] in 1.013s
Matching block [2/4, 3/4] in 0.655s
Matching block [2/4, 4/4] in 0.244s
Matching block [3/4, 1/4] in 0.298s
Matching block [3/4, 2/4] in 0.741s
Matching block [3/4, 3/4] in 0.866s
Matching block [3/4, 4/4] in 0.131s
Matching block [4/4, 1/4] in 0.205s
Matching block [4/4, 2/4] in 0.294s
Matching block [4/4, 3/4] in 0.523s
Matching block [4/4, 4/4] in 0.309s
Elapsed time: 0.116 [minutes]

Loading database

Loading cameras... 174 in 0.000s
Loading matches... 1787 in 0.003s
Loading images... 174 in 0.004s (connected 173)
Building correspondence graph... in 0.017s (ignored 0)

Elapsed time: 0.000 [minutes]


Finding good initial image pair


Initializing with image pair #93 and #88


Global bundle adjustment

iter      cost      cost_change  |gradi

  84  2.086530e+02    3.03e-01    6.08e+03   3.22e+01   6.61e-01  5.82e+05        1    1.42e-03    1.19e-01
  85  2.083633e+02    2.90e-01    5.94e+03   3.07e+01   6.61e-01  6.02e+05        1    1.45e-03    1.21e-01
  86  2.080863e+02    2.77e-01    5.81e+03   2.92e+01   6.60e-01  6.22e+05        1    1.40e-03    1.22e-01
  87  2.078214e+02    2.65e-01    5.67e+03   2.77e+01   6.60e-01  6.43e+05        1    1.40e-03    1.24e-01
  88  2.075681e+02    2.53e-01    5.54e+03   2.62e+01   6.60e-01  6.65e+05        1    1.41e-03    1.25e-01
  89  2.073256e+02    2.42e-01    5.42e+03   2.47e+01   6.59e-01  6.87e+05        1    1.40e-03    1.26e-01
  90  2.070936e+02    2.32e-01    5.29e+03   2.32e+01   6.59e-01  7.10e+05        1    1.40e-03    1.28e-01
  91  2.068716e+02    2.22e-01    5.17e+03   2.18e+01   6.59e-01  7.34e+05        1    1.41e-03    1.29e-01
  92  2.066591e+02    2.13e-01    5.05e+03   2.03e+01   6.59e-01  7.58e+05        1    1.40e-03    1.31e-01
  93  2.064556e+02    2.04e-

...


Bundle adjustment report
------------------------
    Residuals : 5020
   Parameters : 1583
   Iterations : 26
         Time : 0.10586 [s]
 Initial cost : 0.716465 [px]
   Final cost : 0.640918 [px]
  Termination : No convergence

  => Merged observations: 0
  => Completed observations: 11
  => Filtered observations: 121
  => Changed observations: 0.052590

Bundle adjustment report
------------------------
    Residuals : 4800
   Parameters : 1535
   Iterations : 5
         Time : 0.0193861 [s]
 Initial cost : 0.646642 [px]
   Final cost : 0.615595 [px]
  Termination : Convergence

  => Merged observations: 5
  => Completed observations: 27
  => Filtered observations: 1
  => Changed observations: 0.013750

Retriangulation

  => Completed observations: 4
  => Merged observations: 0
  => Retriangulated observations: 0

Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.753621e+03    0.00e+00    2.

   6  3.231360e+03    6.65e-04    7.50e+00   3.52e-01   1.00e+00  7.29e+06        1    1.07e-02    7.56e-02
   7  3.231360e+03    8.30e-05    1.71e+00   6.31e-02   1.01e+00  2.19e+07        1    1.07e-02    8.63e-02
   8  3.231360e+03    6.38e-06    3.37e-01   2.48e-02   1.02e+00  6.56e+07        1    1.07e-02    9.70e-02


Bundle adjustment report
------------------------
    Residuals : 10180
   Parameters : 5308
   Iterations : 9
         Time : 0.097121 [s]
 Initial cost : 0.56499 [px]
   Final cost : 0.563403 [px]
  Termination : Convergence

  => Completed observations: 0
  => Merged observations: 0
  => Filtered observations: 0
  => Changed observations: 0.000000
  => Filtered images: 0

Registering image #165 (14)

  => Image sees 193 / 339 points

Pose refinement report
----------------------
    Residuals : 400
   Parameters : 8
   Iterations : 13
         Time : 0.00383019 [s]
 Initial cost : 0.751453 [px]
   Final cost : 0.65474 [px]
  Termination : Convergence

  => Contin

   3  3.503941e+03    4.32e-02    1.73e+01   4.90e+00   9.97e-01  2.70e+05        1    1.15e-02    4.62e-02
   4  3.503932e+03    9.17e-03    1.29e+01   1.48e+00   1.00e+00  8.10e+05        1    1.15e-02    5.77e-02
   5  3.503929e+03    3.37e-03    1.63e+01   6.84e-01   1.00e+00  2.43e+06        1    1.15e-02    6.92e-02
   6  3.503928e+03    9.08e-04    7.01e+00   2.34e-01   1.01e+00  7.29e+06        1    1.15e-02    8.07e-02
   7  3.503927e+03    1.99e-04    1.73e+00   6.74e-02   1.02e+00  2.19e+07        1    1.15e-02    9.22e-02
   8  3.503927e+03    1.62e-05    3.37e-01   2.17e-02   1.03e+00  6.56e+07        1    1.15e-02    1.04e-01


Bundle adjustment report
------------------------
    Residuals : 10766
   Parameters : 5438
   Iterations : 9
         Time : 0.103714 [s]
 Initial cost : 0.57524 [px]
   Final cost : 0.570493 [px]
  Termination : Convergence

  => Completed observations: 2
  => Merged observations: 0
  => Filtered observations: 0
  => Changed observations: 0.0003

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  3.708931e+03    0.00e+00    3.18e+03   0.00e+00   0.00e+00  1.00e+04        0    7.17e-03    1.19e-02
   1  3.673500e+03    3.54e+01    3.39e+02   5.47e+01   9.96e-01  3.00e+04        1    1.25e-02    2.45e-02
   2  3.672872e+03    6.28e-01    9.20e+01   4.31e+01   1.01e+00  9.00e+04        1    1.21e-02    3.66e-02
   3  3.672848e+03    2.42e-02    1.28e+01   9.53e+00   1.01e+00  2.70e+05        1    1.21e-02    4.86e-02
   4  3.672843e+03    4.91e-03    7.65e+00   1.01e+00   1.00e+00  8.10e+05        1    1.21e-02    6.07e-02
   5  3.672840e+03    2.89e-03    1.30e+01   3.80e-01   9.95e-01  2.43e+06        1    1.21e-02    7.28e-02
   6  3.672838e+03    1.61e-03    2.30e+00   1.83e-01   9.97e-01  7.29e+06        1    1.21e-02    8.49e-02
   7  3.672838e+03    5.25e-04    2.18e+00   2.72e-01   9.93e-01  2.19e+07        1    1.21e-02    9.70e-02
   8  3.672838e+03    5.02e-

  21  1.152712e+04    6.87e-02    2.18e+02   4.14e+00   9.21e-01  2.14e+07        1    1.28e-02    2.59e-01
  22  1.152711e+04    9.32e-03    1.52e+01   1.20e+00   1.00e+00  6.41e+07        1    1.28e-02    2.72e-01
  23  1.152711e+04    1.05e-04    1.21e+00   1.58e-01   1.05e+00  1.92e+08        1    1.28e-02    2.85e-01
  24  1.152711e+04    1.36e-06    2.67e-01   1.31e-02   1.20e+00  5.77e+08        1    1.28e-02    2.98e-01


Bundle adjustment report
------------------------
    Residuals : 11860
   Parameters : 5941
   Iterations : 25
         Time : 0.297777 [s]
 Initial cost : 1.34129 [px]
   Final cost : 0.985866 [px]
  Termination : Convergence

  => Completed observations: 6
  => Merged observations: 0
  => Filtered observations: 57
  => Changed observations: 0.010624

Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  4.203867e+03    0.00e+00    2.08e+04   0.00e+00   0.00e+00  1.00e+04  

In [75]:
len(scene_image_pair_indices)

5646

In [73]:
df_scores = evaluate(df.dropna(subset=['rotation_matrix_prediction', 'translation_vector_prediction']))
df_scores

100%|█████████████████████████████████████████████| 1/1 [00:09<00:00,  9.88s/it]


,dataset,scene,image_pairs,maa,rotation_maa,translation_maa
0,heritage,dioscuri,15051,0.511607,0.651179,0.52071


In [21]:
df_scores.groupby('dataset')[['maa', 'rotation_maa', 'translation_maa']].mean()

,maa,rotation_maa,translation_maa
dataset,,,
haiper,0.966786,0.999405,0.966786
heritage,0.657874,0.748924,0.671365
urban,0.770462,0.875692,0.789539


In [22]:
df_scores.groupby('dataset')[['maa', 'rotation_maa', 'translation_maa']].mean().mean()

maa                0.798374
rotation_maa       0.874674
translation_maa    0.809230
dtype: float32

In [ ]:
df.loc[df['scene'] == 'wall', ['translation_vector', 'translation_vector_prediction']]

In [ ]:
from hloc.utils import viz_3d

In [ ]:
'''

rec_gt = pycolmap.Reconstruction(competition_dataset / 'train' / 'heritage' / 'wall' / 'sfm')

fig = viz_3d.init_figure()
viz_3d.plot_cameras(fig, rec_gt, color='rgba(50,255,50, 0.5)', name="Ground Truth", size=10)
viz_3d.plot_reconstruction(fig, rec_gt, cameras = False, color='rgba(255,50,255, 0.5)', name="Ground Truth", cs=5)
fig.show()
'''

In [ ]:
'''fig = viz_3d.init_figure()
viz_3d.plot_cameras(fig, reconstructions[0], color='rgba(50,255,50, 0.5)', name="Ground Truth", size=10)
viz_3d.plot_reconstruction(fig, reconstructions[0], cameras=False, color='rgba(255,50,255, 0.5)', name="Ground Truth", cs=5)
fig.show()'''